# Data Explorer for SHREC, PPAC, Ruth

In [1]:
!pip install numba
import sys
print(sys.executable)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numba
print(numba.__version__)
import scienceplots
plt.style.use('science')

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [2]:
# Get data
dssd = pd.read_csv('processed_data/dssd_non_vetoed_events.csv') # non-vetoed, calibrated dssd events 
ppac = pd.read_csv('processed_data/ppac_events.csv')
ruth = pd.read_csv('processed_data/rutherford_events.csv')

In [3]:
# DSSD regions
imp = dssd[dssd['event_type'] == 'imp']
boxE = dssd[dssd['event_type'] == 'boxE']
boxW = dssd[dssd['event_type'] == 'boxW']
boxT = dssd[dssd['event_type'] == 'boxT']
boxB = dssd[dssd['event_type'] == 'boxB']

# PPAC 
cathode = ppac[ppac['detector'] == 'cathode']
anodeV = ppac[ppac['detector'] == 'anodeV']
anodeH = ppac[ppac['detector'] == 'anodeH']

# Rutherfords
ruth_E = ruth[ruth['detector'] == 'ruthE']
ruth_W = ruth[ruth['detector'] == 'ruthW']

In [4]:
def print_df_info(df, name):
    print(f"\n{'='*50}")
    print(f"{name.upper()} DATAFRAME SUMMARY")
    print(f"{'='*50}")
    print(f"Total Rows: {len(df)}")
    print("\nColumns:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col:<15} - {df[col].dtype}")
    print(f"\nFirst 5 Rows:\n{df.head()}")

In [5]:
# Print summaries

print_df_info(dssd, "DSSD")
print_df_info(ppac, "PPAC")
print_df_info(ruth, "Rutherford")
print_df_info(imp, "imp")


DSSD DATAFRAME SUMMARY
Total Rows: 1437503

Columns:
 1. t               - float64
 2. x               - int64
 3. y               - int64
 4. tagx            - int64
 5. tagy            - int64
 6. nfile           - int64
 7. xboard          - int64
 8. yboard          - int64
 9. tdelta          - int64
10. nX              - int64
11. nY              - int64
12. xE              - float64
13. yE              - float64
14. event_type      - object

First 5 Rows:
          t    x   y         tagx         tagy  nfile  xboard  yboard  tdelta  \
0  0.001173   37  51   1172537492   1172486495      0       4       6   50997   
1  0.002498  142   7   2498381995   2498317996      0       1       6   63999   
2  0.012275   99   2  12274781996  12274872000      0       2       7  -90004   
3  0.013692   80  13  13691736999  13691683118      0       3       6   53881   
4  0.048685   29  24  48685294463  48685229371      0       4       7   65092   

   nX  nY           xE           yE event_typ

# Correct time offsets for digitizers

In [6]:
# # --- Define manual offsets (in seconds) for each board ---
# manual_offsets = {
#     0: 0,
#     1: -0.045e-6,
#     2: -0.065e-6,
#     3: -0.085e-6,
#     4: -0.105e-6,
#     5: -0.125e-6,
#     6: 0.0,
#     7: 0.0,   # trial offset: add 100 microseconds to board 7
#     8: 0.0
# }

# # --- Recalculate the corrected dt_anodeH (in microseconds) ---
# # Here we use the board information from 'xboard'
# imp['t'] = imp.apply(
#     lambda row: row['t'] + manual_offsets.get(row['xboard'], 0) * 1e6,
#     axis=1
# )

# Rates

In [7]:
# # Need to understand timing first
# plt.figure(figsize=(12, 3))
# plt.subplot(131)
# for event_type, color in zip(['imp', 'boxE', 'boxW', 'boxT', 'boxB'], 
#                               ['red', 'blue', 'green', 'purple', 'orange']):
#     subset = dssd[dssd['event_type'] == event_type]
#     bins = np.linspace(dssd['t'].min(), dssd['t'].max(), num=50)
#     event_counts, _ = np.histogram(subset['t'], bins=bins)
#     time_intervals = np.diff(bins)
#     events_per_second = event_counts / time_intervals
#     plt.plot(bins[:-1], events_per_second, label=event_type, color=color)

# plt.title('DSSD Event Rates by Type')
# plt.xlabel('Time (s)')
# plt.ylabel('Events per Second')
# plt.legend()

# # Event type distribution pie chart
# plt.subplot(132)
# event_counts = dssd['event_type'].value_counts()
# plt.pie(event_counts, labels=event_counts.index, autopct='%1.1f%%')
# plt.title('DSSD Event Type Distribution')

# plt.tight_layout()
# plt.show()



In [8]:
# # Need to understand timing first
# plt.figure(figsize=(12, 3))
# plt.subplot(131)
# for event_type, color in zip(['cathode', 'anodeV', 'anodeH'], 
#                               ['red', 'blue', 'green', 'purple', 'orange']):
#     subset = ppac[ppac['detector'] == event_type]
#     bins = np.linspace(dssd['t'].min(), dssd['t'].max(), num=50)
#     event_counts, _ = np.histogram(subset['t'], bins=bins)
#     time_intervals = np.diff(bins)
#     events_per_second = event_counts / time_intervals
#     plt.plot(bins[:-1], events_per_second, label=event_type, color=color)

# plt.title('PPAC Event Rates by Type')
# plt.xlabel('Time (s)')
# plt.ylabel('Events per Second')
# plt.legend()

# # Event type distribution pie chart
# plt.subplot(132)
# event_counts = ppac['detector'].value_counts()
# plt.pie(event_counts, labels=event_counts.index, autopct='%1.1f%%')
# plt.title('PPAC Event Type Distribution')

# plt.tight_layout()
# plt.show()


In [9]:
# tfile = math.ceil(max(cathode['timetag'])*1e-12)
# plt.hist(cathode['timetag']*1e-12, bins = np.arange(0, tfile, 1), histtype = 'step', color = '#660099', label = 'cathode')
# plt.hist(anodeV['timetag']*1e-12, bins = np.arange(0, tfile, 1), histtype = 'step', color = '#999999', label = 'anode V')
# plt.hist(anodeH['timetag']*1e-12, bins = np.arange(0, tfile, 1), histtype = 'step', color = '#007681', label = 'anode H')
# plt.legend()
# plt.xlabel('time [s]')
# plt.ylabel('rate [Hz]')
# plt.show()

In [10]:
# plt.hist2d(coincident_imp['imp_xE'], coincident_imp['imp_x'], bins=(2000, 174), range=((0,20000),(1,174)), cmin=1)
# plt.show()

In [11]:
# PPAC-IMP Coincidence Analysis using raw timetag values (picoseconds)
# Looking for coincidences around IMP events with window defined in nanoseconds

# Define the coincidence window in nanoseconds
window_before_ns = 5000  # 2000 ns (2 us) before
window_after_ns = 5000   # 1000 ns (1 us) after

# Convert to picoseconds for use with timetag values
window_before_ps = window_before_ns * 1000  # ns → ps
window_after_ps = window_after_ns * 1000    # ns → ps

print(f"Using coincidence window: {window_before_ns} ns before to {window_after_ns} ns after IMP event")
print(f"Window in picoseconds: {window_before_ps} ps before to {window_after_ps} ps after")

# Sort dataframes by timetag for faster searching
cathode_sorted = cathode.sort_values('timetag').reset_index(drop=True)
anodeV_sorted = anodeV.sort_values('timetag').reset_index(drop=True)
anodeH_sorted = anodeH.sort_values('timetag').reset_index(drop=True)
imp_sorted = imp.sort_values('tagx').reset_index(drop=True)  # Using tagx for IMP

# Extract timetag arrays for faster processing
cathode_timetags = cathode_sorted['timetag'].values
anodeV_timetags = anodeV_sorted['timetag'].values
anodeH_timetags = anodeH_sorted['timetag'].values
imp_timetags = imp_sorted['tagx'].values  # Using tagx as the IMP timetag

# Define function to find PPAC events within the time window
def find_events_in_window(imp_timetag, detector_timetags, window_before_ps, window_after_ps):
    """
    Find detector events that occur within a time window around the IMP event.
    All time values are in picoseconds.
    
    Parameters:
    -----------
    imp_timetag : int
        Timestamp of the IMP event in picoseconds
    detector_timetags : array
        Array of detector timestamps in picoseconds
    window_before_ps : int
        Time window before the IMP event in picoseconds
    window_after_ps : int
        Time window after the IMP event in picoseconds
        
    Returns:
    --------
    list
        Indices of events within the window
    """
    # Calculate the time bounds
    lower_bound = imp_timetag - window_before_ps  # Time window before IMP
    upper_bound = imp_timetag + window_after_ps   # Time window after IMP
    
    # Find all events within these bounds using binary search for efficiency
    lower_idx = np.searchsorted(detector_timetags, lower_bound)
    upper_idx = np.searchsorted(detector_timetags, upper_bound)
    
    if upper_idx > lower_idx:
        return list(range(lower_idx, upper_idx))
    return []

# Start timing the process
import time as time_module
start_time = time_module.time()

# Create a list to store coincident events
coincident_events = []

# Number of IMP events to process
total_imp_events = len(imp_sorted)
print(f"Processing {total_imp_events} IMP events...")

# For each IMP event, find coincident PPAC signals
for idx, imp_row in imp_sorted.iterrows():
    imp_timetag = imp_row['tagx']
    
    # Find events in each PPAC detector within the time window
    cathode_indices = find_events_in_window(imp_timetag, cathode_timetags, window_before_ps, window_after_ps)
    anodeV_indices = find_events_in_window(imp_timetag, anodeV_timetags, window_before_ps, window_after_ps)
    anodeH_indices = find_events_in_window(imp_timetag, anodeH_timetags, window_before_ps, window_after_ps)
    
    # Only proceed if we have coincidences in all three PPAC detectors
    if cathode_indices and anodeV_indices and anodeH_indices:
        # Find the closest event in each detector (smallest absolute time difference)
        cathode_diffs = np.abs(cathode_timetags[cathode_indices] - imp_timetag)
        anodeV_diffs = np.abs(anodeV_timetags[anodeV_indices] - imp_timetag)
        anodeH_diffs = np.abs(anodeH_timetags[anodeH_indices] - imp_timetag)
        
        closest_cathode_idx = cathode_indices[np.argmin(cathode_diffs)]
        closest_anodeV_idx = anodeV_indices[np.argmin(anodeV_diffs)]
        closest_anodeH_idx = anodeH_indices[np.argmin(anodeH_diffs)]
        
        # Get the corresponding rows
        cathode_data = cathode_sorted.iloc[closest_cathode_idx]
        anodeV_data = anodeV_sorted.iloc[closest_anodeV_idx]
        anodeH_data = anodeH_sorted.iloc[closest_anodeH_idx]
        
        # Calculate time difference values (in picoseconds)
        # Positive: PPAC after IMP, Negative: PPAC before IMP
        dt_cathode_ps = cathode_data['timetag'] - imp_timetag
        dt_anodeV_ps = anodeV_data['timetag'] - imp_timetag
        dt_anodeH_ps = anodeH_data['timetag'] - imp_timetag
        
        # Create event data dictionary with all relevant information
        event_data = {
            # IMP data
            'imp_timetag': imp_timetag,
            'imp_x': imp_row['x'],
            'imp_y': imp_row['y'],
            'imp_tagx': imp_row['tagx'],
            'imp_tagy': imp_row['tagy'],
            'imp_nfile': imp_row['nfile'],
            'imp_tdelta': imp_row['tdelta'],
            'imp_nX': imp_row['nX'],
            'imp_nY': imp_row['nY'],
            'imp_xE': imp_row['xE'],
            'imp_yE': imp_row['yE'],
            'xboard': imp_row['xboard'],
            'yboard': imp_row['yboard'],
            
            # Cathode data
            'cathode_timetag': cathode_data['timetag'],
            'cathode_energy': cathode_data['energy'],
            'cathode_board': cathode_data['board'],
            'cathode_channel': cathode_data['channel'],
            'cathode_nfile': cathode_data['nfile'],
            
            # AnodeV data
            'anodeV_timetag': anodeV_data['timetag'],
            'anodeV_energy': anodeV_data['energy'],
            'anodeV_board': anodeV_data['board'],
            'anodeV_channel': anodeV_data['channel'],
            'anodeV_nfile': anodeV_data['nfile'],
            
            # AnodeH data
            'anodeH_timetag': anodeH_data['timetag'],
            'anodeH_energy': anodeH_data['energy'],
            'anodeH_board': anodeH_data['board'],
            'anodeH_channel': anodeH_data['channel'],
            'anodeH_nfile': anodeH_data['nfile'],
            
            # Time difference values (in picoseconds)
            'dt_cathode_ps': dt_cathode_ps,
            'dt_anodeV_ps': dt_anodeV_ps,
            'dt_anodeH_ps': dt_anodeH_ps,
            
            # Convert to nanoseconds for convenience
            'dt_cathode_ns': dt_cathode_ps / 1000,
            'dt_anodeV_ns': dt_anodeV_ps / 1000,
            'dt_anodeH_ns': dt_anodeH_ps / 1000
        }
        
        coincident_events.append(event_data)
    
    # Print progress every 10,000 events
    if idx % 10000 == 0 and idx > 0:
        elapsed = time_module.time() - start_time
        events_per_sec = idx / elapsed
        remaining_time = (total_imp_events - idx) / events_per_sec if events_per_sec > 0 else 0
        print(f"Processed {idx}/{total_imp_events} events ({idx/total_imp_events*100:.1f}%) - Rate: {events_per_sec:.1f} events/sec - ETA: {remaining_time:.1f} sec")

# Create the dataframe with coincident events
coincident_imp = pd.DataFrame(coincident_events)
print(f"Found {len(coincident_imp)} coincidences within the window")

# Calculate total processing time
elapsed_time = time_module.time() - start_time
print(f"Total processing time: {elapsed_time:.2f} seconds")
print(f"Processing rate: {total_imp_events/elapsed_time:.1f} events/second")



Using coincidence window: 5000 ns before to 5000 ns after IMP event
Window in picoseconds: 5000000 ps before to 5000000 ps after
Processing 513316 IMP events...
Processed 10000/513316 events (1.9%) - Rate: 9029.7 events/sec - ETA: 55.7 sec
Processed 20000/513316 events (3.9%) - Rate: 9741.0 events/sec - ETA: 50.6 sec
Processed 30000/513316 events (5.8%) - Rate: 10074.2 events/sec - ETA: 48.0 sec
Processed 40000/513316 events (7.8%) - Rate: 10290.1 events/sec - ETA: 46.0 sec
Processed 50000/513316 events (9.7%) - Rate: 10248.3 events/sec - ETA: 45.2 sec
Processed 60000/513316 events (11.7%) - Rate: 10380.2 events/sec - ETA: 43.7 sec
Processed 70000/513316 events (13.6%) - Rate: 10450.0 events/sec - ETA: 42.4 sec
Processed 80000/513316 events (15.6%) - Rate: 10505.6 events/sec - ETA: 41.2 sec
Processed 90000/513316 events (17.5%) - Rate: 10561.0 events/sec - ETA: 40.1 sec
Processed 100000/513316 events (19.5%) - Rate: 10609.4 events/sec - ETA: 39.0 sec
Processed 110000/513316 events (21.

KeyboardInterrupt: 

In [ ]:
# Display time difference statistics in nanoseconds
if not coincident_imp.empty:
    # Convert nanosecond time differences to microseconds for plotting
    coincident_imp['dt_cathode_us'] = coincident_imp['dt_cathode_ns'] / 1000  # ns → μs
    coincident_imp['dt_anodeV_us'] = coincident_imp['dt_anodeV_ns'] / 1000    # ns → μs
    coincident_imp['dt_anodeH_us'] = coincident_imp['dt_anodeH_ns'] / 1000    # ns → μs
    print("\nTime Difference Statistics (ns):")
    for dt_col in ['dt_cathode_ns', 'dt_anodeV_ns', 'dt_anodeH_ns']:
        print(f"{dt_col}:")
        print(f"  Mean: {coincident_imp[dt_col].mean():.2f} ns")
        print(f"  Std Dev: {coincident_imp[dt_col].std():.2f} ns")
        print(f"  Min: {coincident_imp[dt_col].min():.2f} ns")
        print(f"  Max: {coincident_imp[dt_col].max():.2f} ns")
    
    # Count events with PPAC before vs after IMP
    before_count = (coincident_imp['dt_cathode_ns'] < 0).sum()
    after_count = (coincident_imp['dt_cathode_ns'] >= 0).sum()
    print(f"\nPPAC cathode before IMP: {before_count} events ({before_count/len(coincident_imp)*100:.1f}%)")
    print(f"PPAC cathode after IMP: {after_count} events ({after_count/len(coincident_imp)*100:.1f}%)")

    # Create visualization of energy vs time differences
    plt.figure(figsize=(12, 6))

#     # 1. XE vs tof in us
#     plt.subplot(221)
#     plt.scatter(coincident_imp['imp_xE'], coincident_imp['dt_anodeH_us'], 
#                 alpha=0.2, s=1, c='blue')
#     plt.ylabel('Anode H ToF (μs)')
#     plt.xlabel('X-Energy')
#     plt.grid(True, alpha=0.1)
#     plt.ylim(-1.8, -1.3)
#     plt.xlim(0,14000)
#     plt.tight_layout()
    
#     plt.subplot(222)
#     plt.scatter(coincident_imp['imp_xE'], coincident_imp['dt_cathode_us'], 
#                 alpha=0.2, s=1, c='blue')
#     plt.ylabel('Cathode ToF (μs)')
#     plt.xlabel('X-Energy')
#     plt.grid(True, alpha=0.1)
#     plt.ylim(-1.8, -1.3)
#     plt.xlim(0,14000)
#     plt.tight_layout()
    
    plt.subplot(223)
    hexbin = plt.hexbin(coincident_imp['imp_xE'], coincident_imp['dt_anodeH_us'], 
                       gridsize=1400,  # Number of hexagons across the x-direction
                       extent=(0, 14000, -1.8, -1.2),  # (xmin, xmax, ymin, ymax)
                       bins='log',    # Use log scale for counts
                       cmap='viridis',
                       mincnt=1)      # Minimum count to display a hexagon
    plt.xlabel('X-Energy')
    plt.ylabel('Anode H ToF (μs)')
    plt.grid(True, alpha=0.3)
    plt.colorbar(hexbin, label='log10(counts)')
    plt.show()

else:
    print("No coincidences found!")

In [ ]:
# ALpha correlations
print(coincident_imp.head())

In [ ]:
# import numpy as np
# import pandas as pd

# # Sort the imp dataframe by its time column ('t') in ascending order.
# imp = imp.sort_values('t').reset_index(drop=True)

# # Sort the coincident_imp dataframe by the recoil time.
# # Note: recoil time is in picoseconds in the 'imp_timetag' column.
# coincident_imp = coincident_imp.sort_values('imp_timetag').reset_index(drop=True)


# # --- Parameters for the alpha decay time window ---
# # Define the window in seconds and convert to microseconds (1 s = 1e6 µs)
# alpha_min_time_s = 0.1     # Minimum time after recoil to consider (in seconds)
# alpha_max_time_s = 10    # Maximum time after recoil (in seconds)
# alpha_min_us = alpha_min_time_s * 1e6
# alpha_max_us = alpha_max_time_s * 1e6

# # --- Build pixel history from the imp dataframe ---
# # Group the imp dataframe by pixel coordinates (x, y).
# # The imp dataframe is assumed to be time-ordered based on the 't' column.
# pixel_groups = imp.groupby(['x', 'y'])

# # Build a dictionary mapping each pixel (x, y) to its corresponding time-ordered DataFrame.
# pixel_history = {pixel: group for pixel, group in pixel_groups}

# # --- Find candidate alpha events ---
# # The coincident_imp dataframe contains recoil events.
# # For each recoil event, we'll search for subsequent events in the same pixel from imp.
# alpha_candidates = []  # This list will store DataFrames of candidate alpha events.

# for recoil_idx, recoil in coincident_imp.iterrows():
#     # Determine the pixel for the recoil event using the 'imp_x' and 'imp_y' columns.
#     pixel = (recoil['imp_x'], recoil['imp_y'])
    
#     # Convert the recoil's imp_timetag from picoseconds to microseconds.
#     recoil_time_us = recoil['imp_timetag'] / 1e6
    
#     # Check if there are any events in the same pixel in the imp dataframe.
#     if pixel not in pixel_history:
        
#         continue  # Skip if no events are found for this pixel.
    
#     # Retrieve the time-ordered events for this pixel.
#     pixel_df = pixel_history[pixel]
    
#     # Get the 't' values (in µs) as a sorted NumPy array.
#     time_array = pixel_df['t'].values
    
#     # Define the lower and upper bounds for candidate alpha events.
#     lower_bound = recoil_time_us + alpha_min_us
#     upper_bound = recoil_time_us + alpha_max_us
    
#     # Use binary search to find the index positions in the time_array.
#     start_idx = np.searchsorted(time_array, lower_bound, side='left')
#     end_idx = np.searchsorted(time_array, upper_bound, side='right')
    
#     # If events exist in the window, add them as candidate alpha events.
#     if start_idx < end_idx:
#         candidate_events = pixel_df.iloc[start_idx:end_idx].copy()
#         # Record the associated recoil information for later reference.
#         candidate_events['recoil_index'] = recoil_idx
#         candidate_events['recoil_time_us'] = recoil_time_us
#         alpha_candidates.append(candidate_events)

# # Combine all candidate alpha events into a single DataFrame.
# if alpha_candidates:
#     alpha_candidates_df = pd.concat(alpha_candidates, ignore_index=True)
# else:
#     alpha_candidates_df = pd.DataFrame()

# # Display the first few candidate alpha events.
# print(alpha_candidates_df.head())


In [ ]:
import numpy as np
import pandas as pd

# --- Ensure time-ordering of dataframes ---
# imp: Implant events (t is in seconds)
imp = imp.sort_values('t').reset_index(drop=True)
# coincident_imp: Recoil events (imp_timetag is in picoseconds)
coincident_imp = coincident_imp.sort_values('imp_timetag').reset_index(drop=True)

# --- Parameters for the alpha decay time window (in seconds) ---
alpha_min_time_s = 0.0001     # Minimum time after recoil to consider (in seconds)
alpha_max_time_s = 7.7     # Maximum time after recoil to consider (in seconds)

# --- Build pixel history from the imp dataframe ---
# Group the full implant event history by pixel coordinates (x, y)
pixel_groups = imp.groupby(['x', 'y'])
pixel_history = {pixel: group for pixel, group in pixel_groups}

# --- Find candidate alpha events ---
# The coincident_imp dataframe contains recoil events.
# For each recoil event, we'll search for subsequent events in the same pixel from imp.
alpha_candidates = []  # List to store DataFrames of candidate alpha events

for recoil_idx, recoil in coincident_imp.iterrows():
    # Determine the pixel for the recoil event using 'imp_x' and 'imp_y'
    pixel = (recoil['imp_x'], recoil['imp_y'])
    
    # Convert the recoil's imp_timetag from picoseconds to seconds.
    recoil_time_sec = recoil['imp_timetag'] / 1e12
    
    # Check if there are any events in the same pixel in the imp dataframe.
    if pixel not in pixel_history:
        continue  # Skip if no events are found for this pixel.
    
    # Retrieve the time-ordered events for this pixel from imp.
    pixel_df = pixel_history[pixel]
    
    # Get the 't' values (in seconds) as a sorted NumPy array.
    time_array = pixel_df['t'].values
    
    # Define the lower and upper bounds for candidate alpha events in seconds.
    lower_bound = recoil_time_sec + alpha_min_time_s
    upper_bound = recoil_time_sec + alpha_max_time_s
    
    # Use binary search to find the index positions in the time_array.
    start_idx = np.searchsorted(time_array, lower_bound, side='left')
    end_idx = np.searchsorted(time_array, upper_bound, side='right')
    
    # If events exist in the window, add them as candidate alpha events.
    if start_idx < end_idx:
        candidate_events = pixel_df.iloc[start_idx:end_idx].copy()
        # Record the associated recoil information for later reference.
        candidate_events['recoil_index'] = recoil_idx
        candidate_events['recoil_time_sec'] = recoil_time_sec
        alpha_candidates.append(candidate_events)

# Combine all candidate alpha events into a single DataFrame.
if alpha_candidates:
    alpha_candidates_df = pd.concat(alpha_candidates, ignore_index=True)
else:
    alpha_candidates_df = pd.DataFrame()

# Display the first few candidate alpha events.
print(alpha_candidates_df.head())


In [ ]:
alpha_candidates_df['log_dt'] = np.log(abs(alpha_candidates_df['t'] - alpha_candidates_df['recoil_time_sec']))

In [ ]:
plt.figure(figsize=(8,4))
plt.scatter(alpha_candidates_df['yE'], alpha_candidates_df['log_dt'], alpha=0.1)
plt.xlim(0,14000)
plt.show()

In [ ]:
plt.figure(figsize=(6,3))
plt.hist2d(alpha_candidates_df['yE'], alpha_candidates_df['log_dt'], 
           bins=((500),(50)), range=((5000,10000),(-3,3)), cmin=1)
plt.xlabel('Event Energy (keV))')
plt.ylabel(r'log(dt/s)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Map dt_anodeH_us from coincident_imp to alpha candidates using the recoil_index.
# Make sure coincident_imp is indexed by its default integer index.
alpha_candidates_df['dt_anodeH_us'] = alpha_candidates_df['recoil_index'].map(coincident_imp.set_index(coincident_imp.index)['dt_anodeH_us'])

# Filter recoils to only those that have an associated alpha candidate.
recoil_indices_with_alpha = alpha_candidates_df['recoil_index'].unique()
recoils_with_alpha = coincident_imp.loc[recoil_indices_with_alpha]

plt.figure(figsize=(5,3))
plt.scatter(coincident_imp['imp_xE'], coincident_imp['dt_anodeH_us'], 
            alpha=0.01, s=1, c='blue')
plt.scatter(recoils_with_alpha['imp_xE'], recoils_with_alpha['dt_anodeH_us'], 
            alpha=0.1, s=1, c='r')
plt.ylabel('Anode H ToF (us)')
plt.xlabel('X-Energy')
plt.grid(True, alpha=0.1)
plt.ylim(-1.8, -1.3)
plt.xlim(0,14000)
plt.tight_layout()


plt.xlabel('X-Energy (keV)')
plt.ylabel('Time Difference (us)')
plt.xlim(0, 14000)
plt.ylim(-1.8,-1.25)
plt.legend()
plt.show()


In [ ]:
# --- Set an energy condition on the candidate alphas ---
energy_min = 8100    # Minimum energy (keV)
energy_max = 8400   # Maximum energy (keV)

# Filter the alpha_candidates_df using the energy condition on xE.
filtered_alpha_candidates = alpha_candidates_df[(alpha_candidates_df['xE'] >= energy_min) & 
                                                (alpha_candidates_df['xE'] <= energy_max)].copy()

# Map dt_anodeH_us from coincident_imp to the candidate alphas using recoil_index.
# Here, we assume coincident_imp is indexed by its default integer index.
filtered_alpha_candidates['dt_anodeH_us'] = filtered_alpha_candidates['recoil_index'].map(
    coincident_imp.set_index(coincident_imp.index)['dt_anodeH_us'])

# Filter recoil events to only those that have an associated alpha candidate.
recoil_indices_with_alpha = filtered_alpha_candidates['recoil_index'].unique()
recoils_with_alpha = coincident_imp.loc[recoil_indices_with_alpha]

plt.figure(figsize=(5,3))
plt.scatter(filtered_alpha_candidates['xE'], filtered_alpha_candidates['dt_anodeH_us'],
            s=10, color='red', alpha=0.7, label='Alpha Candidates (Energy Condition)')

plt.xlabel('X-Energy (keV)')
plt.ylabel('Time Difference (us)')
plt.xlim(0, 14000)
plt.legend()
plt.show()

In [ ]:

filtered_alpha_candidates['dt_anodeH_us'] = filtered_alpha_candidates['recoil_index'].map(
    coincident_imp.set_index(coincident_imp.index)['dt_anodeH_us']
)

print(filtered_alpha_candidates.head())

# Filter recoils: extract the unique recoil indices from the energy-filtered candidate alphas.
recoil_indices_with_alpha_filtered = filtered_alpha_candidates['recoil_index'].unique()
recoils_with_alpha_filtered = coincident_imp.loc[recoil_indices_with_alpha_filtered]


# Set energy gate for recoils now:
energy_lower_bound = 2000
energy_upper_bound = 8099
recoils_with_alpha_filtered = recoils_with_alpha_filtered[
    (recoils_with_alpha_filtered['imp_xE'] >= energy_lower_bound) &
    (recoils_with_alpha_filtered['imp_xE'] <= energy_upper_bound)
]

plt.figure(figsize=(10,6))

plt.subplot(221)
plt.scatter(coincident_imp['imp_xE'], coincident_imp['dt_anodeH_us'], 
            alpha=0.01, s=1, c='blue', label='')
# Overlay the recoil events that have an energy-filtered alpha candidate (red)
plt.scatter(recoils_with_alpha_filtered['imp_xE'], recoils_with_alpha_filtered['dt_anodeH_us'], 
            alpha=0.4, s=1, c='red', label='alpha tagged')
plt.xlabel('X-Energy (keV)')
plt.ylabel('Anode H ToF (us)')
plt.xlim(0,14000)
plt.ylim(-1.7,-1.25)
plt.grid(True, alpha=0.1)
plt.legend()

plt.subplot(222)
plt.hist2d(alpha_candidates_df['yE'], alpha_candidates_df['log_dt'], 
           bins=((500),(50)), range=((5000,10000),(-3,3)), cmin=1)
plt.fill_betweenx(y=[np.log(alpha_min_time_s), np.log(alpha_max_time_s)], x1=energy_min, x2=energy_max, 
                  color='red', alpha=0.2, label='Energy Gate')
plt.xlabel('Event Energy (keV))')
plt.ylabel(r'log(dt/s)')

plt.subplot(223)
plt.hist2d(recoils_with_alpha_filtered['imp_x'], recoils_with_alpha_filtered['imp_y'], 
           bins=((174),(59)), range=((-1,174),(-1,60)), cmin=1)
plt.xlabel('X strip')
plt.ylabel(r'y strip')




plt.tight_layout()
plt.show()

In [ ]:
print(coincident_imp.head())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
import pandas as pd

# --- Ensure a time column exists in coincident_imp ---
if 't' not in coincident_imp.columns:
    coincident_imp['t'] = coincident_imp['imp_timetag'] * 1e-12

# --- Define manual offsets (in seconds) for each board ---
manual_offsets = {
    0: 0,
    1: -0.045e-6,
    2: -0.065e-6,
    3: -0.085e-6,
    4: -0.105e-6,
    5: -0.125e-6,
}

# manual_offsets = {
#     0: 0,
#     1: 0,
#     2: 0,
#     3: 0,
#     4: 0,
#     5: 0,
#     6: 0.0,
#     7: 0.0,   # trial offset: add 100 microseconds to board 7
#     8: 0.0
# }

# --- Recalculate the corrected dt_anodeH (in microseconds) ---
# Here we use the board information from 'xboard'
coincident_imp['dt_anodeH_us_corr'] = coincident_imp.apply(
    lambda row: row['dt_anodeH_us'] + manual_offsets.get(row['xboard'], 0) * 1e6,
    axis=1
)

# --- Plotting: Recoil Energy vs. Corrected dt_anodeH with board-specific colors ---

plt.figure(figsize=(8,4))
fs=18

# Get a list of unique boards
boards = sorted(coincident_imp['xboard'].unique())

# Create a colormap (one color per board)
colors = plt.cm.tab10(np.linspace(0, 1, len(boards)))
# colors = ['r', 'g', 'b', 'purple', 'gray', 'pink', 'olive']
# alpha_board = 0.3
legend_handles = []
# Loop over boards and plot each in a different color
for board, color in zip(boards, colors):
#     alpha = alpha_board - 0.05
    # Filter the DataFrame for this board
    board_data = coincident_imp[coincident_imp['xboard'] == board]
    plt.scatter(board_data['imp_xE'], board_data['dt_anodeH_us'],
                s=2, alpha=0.1, color=color, label=f'Board {board}')
    # Create a proxy artist for the legend with full opacity
    legend_handles.append(Line2D([0], [0], marker='o', color='w', markersize=6, markerfacecolor=color, label=f'Board {board}'))


plt.xlabel("SHREC implant energy (keV)", fontsize=fs)
plt.ylabel(r"ToF ($\mu$s)", fontsize=fs)
plt.title("E-ToF by implant board", fontsize=fs+2)
plt.xlim(0, 14000)
plt.ylim(-1.7, -1.35)
plt.grid(True, alpha=0.3)
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=fs-2)
plt.legend(handles=legend_handles, fontsize=fs-4, frameon=True)
plt.savefig("plots/etof_by_board.pdf", dpi=1000)


In [ ]:
print(len(recoils_with_alpha_filtered))
print(imp.head())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# --- Step 1: Ensure a time column exists in filtered_alpha_candidates ---
# (Convert imp_timetag from picoseconds to seconds if needed.)
if 't' not in filtered_alpha_candidates.columns:
    recoils_with_alpha_filtered['t'] = recoils_with_alpha_filtered['imp_timetag'] * 1e-12

# --- Step 2: Define manual offsets (in seconds) for each board ---
# Adjust these values by trial and error; for example, here we set specific values.
manual_offsets = {
    0: 0.0,
    1: -0.2-6,
    2: -0.065e-6,
    3: -0.085e-6,
    4: -0.105e-6,
    5: -0.125e-6,
    6: 0.0,
    7: 0.0,
    8: 0.0
}

# --- Step 3: Recalculate the corrected dt_anodeH (in microseconds) ---
# For each row in filtered_alpha_candidates, add the manual offset (converted to μs)
# based on its xboard.
recoils_with_alpha_filtered['dt_anodeH_us_corr'] = recoils_with_alpha_filtered.apply(
    lambda row: row['dt_anodeH_us'] + manual_offsets.get(row['xboard'], 0) * 1e6,
    axis=1
)

# Optionally, inspect the first few rows to verify:
print(recoils_with_alpha_filtered[['dt_anodeH_us', 'dt_anodeH_us_corr', 'xboard', 'imp_xE']].head())

# --- Step 4: Plot the ETOF for Filtered Alpha Candidates ---
plt.figure(figsize=(6,3))
fs=16
# plt.subplot(221)
# plt.scatter(coincident_imp['imp_xE'], coincident_imp['dt_anodeH_us_corr'], 
#             alpha=0.01, s=4, c='blue', label='')
# plt.scatter(recoils_with_alpha_filtered['imp_xE'], recoils_with_alpha_filtered['dt_anodeH_us_corr'],
#             s=4, alpha=0.5, color='r', label='Filtered Alpha Candidates (Corrected)')
# plt.xlabel("Recoil Energy (keV)")
# plt.ylabel("Corrected dt_anodeH (μs)")
# plt.title("ETOF: Recoil Energy vs. Corrected dt_anodeH for Filtered Alpha Candidates")
# plt.xlim(0, 14000)
# plt.ylim(-1.65, -1.5)
# plt.grid(True, alpha=0.3)
# plt.legend()

# plt.subplot(122)
plt.hist2d(coincident_imp['imp_xE'], coincident_imp['dt_anodeH_us_corr'], 
           bins=(300, 100), range=((0, 15000), (-1.7, -1.5)), cmin=1, cmap='viridis',
          )
plt.scatter(recoils_with_alpha_filtered['imp_xE'], recoils_with_alpha_filtered['dt_anodeH_us_corr'],
            color='red', alpha=0.4,s=20, label=r'$\alpha$-tagged')  # s is the marker size
# plt.colorbar()  # Optionally, add a colorbar to see the mapping
plt.ylim(-1.625, -1.50)
plt.xlabel('SHREC implant energy (keV)',fontsize=fs)
plt.ylabel(r'ToF ($\mu$s/ 10keV)', fontsize=fs)
ax = plt.gca()  # Get the current axes
ax.tick_params(axis='both', which='major', labelsize=fs-2)  # Adjust tick label size for both axes
plt.legend(loc='lower right', fontsize=fs-2, frameon=True)
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

plt.figure(figsize=(8,4))
fs = 18

# Hexbin plot
plt.hexbin(coincident_imp['imp_xE'], coincident_imp['dt_anodeH_us_corr'], 
           gridsize=200, extent=(0, 10000, -1.7, -1.5), mincnt=1.5, cmap='viridis', alpha=0.9)
# plt.colorbar(alpha=1)

# Scatter plot with lower alpha
plt.scatter(recoils_with_alpha_filtered['imp_xE'], recoils_with_alpha_filtered['dt_anodeH_us_corr'],
            color='red', alpha=0.4, s=20, label=r'$\alpha$-tagged')  # Overlay sparse events

# Create a proxy legend marker with full opacity
legend_marker = Line2D([0], [0], marker='o', color='w', markersize=6, 
                        markerfacecolor='red', label=r'$\alpha$-tagged')

plt.ylim(-1.625, -1.49)
plt.xlim(0, 10000)

plt.xlabel('SHREC implant energy (keV)', fontsize=fs)
plt.ylabel(r'ToF ($\mu$s/ 50keV)', fontsize=fs)
plt.title(r'$\alpha$-correlated E-ToF', fontsize=fs+2)


ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=fs-2)
plt.legend(handles=[legend_marker], loc='lower right', fontsize=fs-2, frameon=True)
plt.savefig('plots/correlated_etof.pdf', dpi=300)
